In [ ]:
import pandas as pd
import rdkit
from rdkit import DataStructs, Chem
from rdkit.Chem import MACCSkeys, Draw
import torch
from fast_jtnn import *
from rdkit import RDLogger

# Disable RDKit warning messages
RDLogger.DisableLog('rdApp.*')


In [ ]:
from fast_molvae.sample import load_model
model = load_model('./data/vocab.txt', './fast_molvae/vae_model/model.epoch-19')
vocab = [x.strip("\r\n ") for x in open('./data/vocab.txt')]
vocab = Vocab(vocab)

Generate Sample

In [ ]:
smi_target = ['c1ccccc1']
decode_smiles = set()
step_size = 0.01
generate_samples_count = 5
tree_batch = [MolTree(s) for s in smi_target]
_, jtenc_holder, mpn_holder = datautils.tensorize(tree_batch, vocab, assm=False)
tree_vecs, _, mol_vecs = model.encode(jtenc_holder, mpn_holder)
count = 0
while len(decode_smiles) < generate_samples_count:
    count += 1
    z_tree_mean = model.T_mean(tree_vecs)
    epsilon_tree = create_var(torch.randn_like(z_tree_mean))
    z_tree_log_var = -torch.abs(model.T_var(tree_vecs))
    z_mol_mean = model.G_mean(mol_vecs)
    z_mol_log_var = -torch.abs(model.G_var(mol_vecs))
    z_tree_mean_new = z_tree_mean + epsilon_tree * step_size
    epsilon_mol = create_var(torch.randn_like(z_mol_mean))
    z_mol_mean_new = z_tree_mean + epsilon_mol * step_size
    decode_smiles.add(model.decode(z_tree_mean_new, z_mol_mean_new, False))
    if count > generate_samples_count :
        step_size += 0.05
        count = 0
        print('Increase step size !')
img = Draw.MolsToGridImage([Chem.MolFromSmiles(mol) for mol in decode_smiles], molsPerRow=5,subImgSize=(500,500), legends=decode_smiles)
img